ME204 - Data Engineering for the Social World

# **Notebook 2 - Database Building**

Goals:
- Reorganize columns (price, course, provider) and rows (NA's) to filter correct information.
- Remove any courses that were "parent" pages leading to more classes I already scraped, as this is redundant data and affects eventual EDA.
- Link my DataFrame to a SQLite database, run queries to discover insights for questions I have
    - Ex. Examining the relationship between course duration and price: which courses offer the 'best' deal?
    - Ex. Which categories have the highest enrollment?

Imports:

In [12]:
#For data cleaning
import pandas as pd
import os
import sqlite3
from datetime import datetime
import re

Reading the CSV into a DataFrame:

In [15]:
os.getcwd()
os.chdir("../myproject/data/raw")

'/Users/vaughnmitchell/Desktop/LSE/ME204/code/myproject/data/raw'

In [16]:
df_uncleaned = pd.read_csv("raw.csv")
df_uncleaned.head()

,Unnamed: 0,course,provider,link,course_length,time_commitment,start_date,current_enrollment,pace,price,subject
0,0,How to Learn Online,edX,https://www.edx.org/learn/how-to-learn/edx-how...,2 weeks,2–3 hours per week,Starts Jul 27,287998,Self-paced,£54,Education & Teacher Training
1,1,The Science of\nHappiness,"University of California, Ber…",https://www.edx.org/learn/happiness/university...,11 weeks,4–5 hours per week,Starts Jul 27,583920,Self-paced,£131,Social Sciences
2,2,Remote Work\nRevolution for\nEveryone,Harvard University,https://www.edx.org/learn/remote-work/harvard-...,3 weeks,2–3 hours per week,Starts Jul 27,114666,Self-paced,£116,Business & Management
3,3,CS50's Introduction to\nComputer Science,Harvard University,https://www.edx.org/learn/computer-science/har...,12 weeks,6–18 hours per week,Starts Jul 27,5988868,Self-paced,£0,Computer Science
4,4,Data Visualization and\nBuilding Dashboards\nw...,IBM,https://www.edx.org/learn/data-visualization/i...,4 weeks,2–3 hours per week,Starts Jul 27,48226,Self-paced,£77,Data Analysis & Statistics


When we saved the data to a CSV, a new column called "Unnamed: 0" was created. Let's delete that first:

In [17]:
df_uncleaned = df_uncleaned.drop("Unnamed: 0",axis=1)

In [18]:
df_uncleaned.head()

,course,provider,link,course_length,time_commitment,start_date,current_enrollment,pace,price,subject
0,How to Learn Online,edX,https://www.edx.org/learn/how-to-learn/edx-how...,2 weeks,2–3 hours per week,Starts Jul 27,287998,Self-paced,£54,Education & Teacher Training
1,The Science of\nHappiness,"University of California, Ber…",https://www.edx.org/learn/happiness/university...,11 weeks,4–5 hours per week,Starts Jul 27,583920,Self-paced,£131,Social Sciences
2,Remote Work\nRevolution for\nEveryone,Harvard University,https://www.edx.org/learn/remote-work/harvard-...,3 weeks,2–3 hours per week,Starts Jul 27,114666,Self-paced,£116,Business & Management
3,CS50's Introduction to\nComputer Science,Harvard University,https://www.edx.org/learn/computer-science/har...,12 weeks,6–18 hours per week,Starts Jul 27,5988868,Self-paced,£0,Computer Science
4,Data Visualization and\nBuilding Dashboards\nw...,IBM,https://www.edx.org/learn/data-visualization/i...,4 weeks,2–3 hours per week,Starts Jul 27,48226,Self-paced,£77,Data Analysis & Statistics


I also see that the `price` column has the "£" sign in it. Also, there are some prices that have a "," in them. I'd like to remove both symbols:

In [19]:
df_uncleaned['price'] = df_uncleaned['price'].str.replace("£","").str.replace(",","").astype(int)

Lastly with the `price` column, I should mention that all of these courses are *technically* free. However, my EDA is concerned with courses that offer a certificate, so I will remove all rows that contain a "0" as no certificate is available:

In [21]:
df_uncleaned = df_uncleaned[df_uncleaned['price'] != 0]

In [22]:
df_uncleaned.head()

,course,provider,link,course_length,time_commitment,start_date,current_enrollment,pace,price,subject
0,How to Learn Online,edX,https://www.edx.org/learn/how-to-learn/edx-how...,2 weeks,2–3 hours per week,Starts Jul 27,287998,Self-paced,54,Education & Teacher Training
1,The Science of\nHappiness,"University of California, Ber…",https://www.edx.org/learn/happiness/university...,11 weeks,4–5 hours per week,Starts Jul 27,583920,Self-paced,131,Social Sciences
2,Remote Work\nRevolution for\nEveryone,Harvard University,https://www.edx.org/learn/remote-work/harvard-...,3 weeks,2–3 hours per week,Starts Jul 27,114666,Self-paced,116,Business & Management
4,Data Visualization and\nBuilding Dashboards\nw...,IBM,https://www.edx.org/learn/data-visualization/i...,4 weeks,2–3 hours per week,Starts Jul 27,48226,Self-paced,77,Data Analysis & Statistics
5,"Six Sigma Part 2:\nAnalyze, Improve,\nControl",Technische Universität Mün…,https://www.edx.org/learn/six-sigma/technische...,8 weeks,3–4 hours per week,Starts Jul 27,85904,Self-paced,147,Business & Management


Let's also add the '£' symbol back into the name of the price column to remember what currency we are working with:

In [23]:
df_uncleaned = df_uncleaned.rename(columns={"price":"price_£"})

In [24]:
df_uncleaned.columns

Index(['course', 'provider', 'link', 'course_length', 'time_commitment',
       'start_date', 'current_enrollment', 'pace', 'price_£', 'subject'],
      dtype='object')

Now take a look at the `course` column. For some courses that span multiple lines, there is a "\n" that needs to be removed:

In [25]:
df_uncleaned['course'] = df_uncleaned['course'].str.replace("\n", " ")

Likewise with the `provider` column, some university/insitution names span multiple lines, there is a "\n" that needs to be removed. Removing the "\n" from both columns could be done in one line, but for instructive purposes I run the commands separately:

In [26]:
df_uncleaned['provider'] = df_uncleaned['provider'].str.replace("\n" , " ")

In [27]:
df_uncleaned.head()

,course,provider,link,course_length,time_commitment,start_date,current_enrollment,pace,price_£,subject
0,How to Learn Online,edX,https://www.edx.org/learn/how-to-learn/edx-how...,2 weeks,2–3 hours per week,Starts Jul 27,287998,Self-paced,54,Education & Teacher Training
1,The Science of Happiness,"University of California, Ber…",https://www.edx.org/learn/happiness/university...,11 weeks,4–5 hours per week,Starts Jul 27,583920,Self-paced,131,Social Sciences
2,Remote Work Revolution for Everyone,Harvard University,https://www.edx.org/learn/remote-work/harvard-...,3 weeks,2–3 hours per week,Starts Jul 27,114666,Self-paced,116,Business & Management
4,Data Visualization and Building Dashboards wit...,IBM,https://www.edx.org/learn/data-visualization/i...,4 weeks,2–3 hours per week,Starts Jul 27,48226,Self-paced,77,Data Analysis & Statistics
5,"Six Sigma Part 2: Analyze, Improve, Control",Technische Universität Mün…,https://www.edx.org/learn/six-sigma/technische...,8 weeks,3–4 hours per week,Starts Jul 27,85904,Self-paced,147,Business & Management


There could also be issues with the `course_length` column as the type is a string. I want to remove the " weeks" information from the entries and only extract the number as an integer, renaming the column to still understand that the data is in weeks:

In [28]:
df_uncleaned['course_length'] = df_uncleaned['course_length'].str.replace(" weeks", "").astype(int)

Now let's rename the column:

In [29]:
df_uncleaned = df_uncleaned.rename(columns={"course_length": "course_length_weeks"})

In [30]:
df_uncleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 954 entries, 0 to 999
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   course               954 non-null    object
 1   provider             954 non-null    object
 2   link                 954 non-null    object
 3   course_length_weeks  954 non-null    int64 
 4   time_commitment      954 non-null    object
 5   start_date           953 non-null    object
 6   current_enrollment   954 non-null    int64 
 7   pace                 954 non-null    object
 8   price_£              954 non-null    int64 
 9   subject              954 non-null    object
dtypes: int64(3), object(7)
memory usage: 82.0+ KB


I also see that some courses are available from today until December 31st. This means that we have to rescrape the data every day because the start date would update online but not in my DataFrame. Let me change this:

In [31]:
df_uncleaned['start_date'] = df_uncleaned['start_date'].str.replace("Starts","").str.strip()
df_uncleaned['start_date'] = df_uncleaned['start_date'].str.replace("Started","").str.strip()
df_uncleaned['start_date'] = df_uncleaned['start_date'] + ' 2024'

In [32]:
df_uncleaned['start_date'] = pd.to_datetime(df_uncleaned['start_date'], format='%b %d %Y')

There is at least one entry with "NaT" or "Not a Time":

In [33]:
df_uncleaned = df_uncleaned.dropna(subset='start_date')

In [34]:
today = datetime.today()
def replace_with_today(date):
    if date.date() == today.date():
        return 'Available Today'
    else:
        return date.strftime('%b %d')
df_uncleaned['start_date'] = df_uncleaned['start_date'].apply(replace_with_today)

In [35]:
df_uncleaned.head()

,course,provider,link,course_length_weeks,time_commitment,start_date,current_enrollment,pace,price_£,subject
0,How to Learn Online,edX,https://www.edx.org/learn/how-to-learn/edx-how...,2,2–3 hours per week,Jul 27,287998,Self-paced,54,Education & Teacher Training
1,The Science of Happiness,"University of California, Ber…",https://www.edx.org/learn/happiness/university...,11,4–5 hours per week,Jul 27,583920,Self-paced,131,Social Sciences
2,Remote Work Revolution for Everyone,Harvard University,https://www.edx.org/learn/remote-work/harvard-...,3,2–3 hours per week,Jul 27,114666,Self-paced,116,Business & Management
4,Data Visualization and Building Dashboards wit...,IBM,https://www.edx.org/learn/data-visualization/i...,4,2–3 hours per week,Jul 27,48226,Self-paced,77,Data Analysis & Statistics
5,"Six Sigma Part 2: Analyze, Improve, Control",Technische Universität Mün…,https://www.edx.org/learn/six-sigma/technische...,8,3–4 hours per week,Jul 27,85904,Self-paced,147,Business & Management


Later in our EDA and visualization, there could be a potential problem with the `time_commitment` column as it is a string and not an integer. Let's define a function that finds the midpoint of these hourly ranges and casts them to an integer:

In [36]:
def extract_average(s):
    numbers = list(map(int, re.findall(r'\d+', s)))
    if len(numbers) == 2:
        return (numbers[0] + numbers[1]) / 2

# Apply the function to the 'time_commitment' column
df_uncleaned['time_commitment'] = df_uncleaned['time_commitment'].apply(extract_average)

Finally, there are some "NA" values that were collected in the `start_date`, `current_enrollment`,`pace`, and `subject` columns. This was due to the fact that the webdriver found a page and collected values it could find for a course "parent" page, which means an institution had a another subpage for additional classes it offered. This does not mean data was lost, as I still scraped these "child" courses in the original script. Let's remove any institutions' courses that led to a parent course page on the edX site:

In [37]:
columns_to_check = ['start_date','current_enrollment','pace','subject']
df_uncleaned = df_uncleaned.dropna(axis=0, subset=columns_to_check)

Now, `df_uncleaned` seems tidy. Here I will save it to another DataFrame `df_clean` and export that to a SQLite database:

In [38]:
df_clean = df_uncleaned


In [49]:
# os.chdir("../data/clean")
DATA_FOLDER = os.getcwd()


Now, let me load the SQL extension into this notebook and configure autocommits:

In [50]:
%load_ext sql
%config SqlMagic.autocommit=True

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


Creating a SQLite3 database:

In [51]:
conn = sqlite3.connect(os.path.join(DATA_FOLDER, "edx.db"))

Creating our our first table:

In [52]:
conn.execute('''
CREATE TABLE IF NOT EXISTS courses(
    course TEXT PRIMARY KEY,
    provider TEXT,
    link TEXT,
    course_length_weeks INTEGER,
    time_commitment TEXT,
    start_date TEXT,
    current_enrollment INTEGER,
    pace TEXT,
    price_£ INTEGER,
    subject TEXT     
    )   
''')

Here I export the `df_clean` DataFrame to the SQLite `courses` table, which essentially copies all the data from our cleaned DataFrame into one master table:

In [53]:
df_clean.to_sql('courses', conn, if_exists="replace", index=False)


953

I am also interested in the relationship between course length and price to see which courses offer the high duration-to-value ratio:

In [54]:
conn.execute('''
CREATE TABLE IF NOT EXISTS value (
    course TEXT PRIMARY KEY,
    course_length_weeks INTEGER,
    price_£ INTEGER
)
''')

df_clean[['course','course_length_weeks','price_£']].to_sql('value', conn, if_exists="replace", index=False)

953

Now, let's make a table that can help us examine enrollment by category:

In [55]:
conn.execute('''
CREATE TABLE IF NOT EXISTS enrollment_by_subject(
    course TEXT PRIMARY KEY,
    current_enrollment INTEGER,
    subject TEXT          
)
''')

df_clean[['course','current_enrollment','subject']].to_sql('enrollment_by_subject', conn, if_exists="replace", index=False)

953

This marks the end of this notebook. See `NB03-EDA-and-Dataviz.ipynb` for exploratory data analysis and data cleaning.